## 1. Extraction of sentence

In [1]:
import pandas as pd
import Extractor
import sqlite3
df = pd.read_excel('들,2,3_0.xlsx')
new_words = df[df.y == 1].iloc[:,1].drop_duplicates(keep='first')

FileNotFoundError: [Errno 2] No such file or directory: '들,2,3_0.xlsx'

In [2]:
conn = sqlite3.connect('Total_Ent.db')
df1 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
conn = sqlite3.connect('HP.db')
df2 = pd.read_sql('SELECT head FROM head', conn)
conn.close()

In [3]:
head = df1.append(df2)

In [4]:
ext = Extractor.Ext(head)
head = ext.cleaning()

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107


In [5]:
sent = ext.extract_sent(new_words)

In [25]:
sent = pd.DataFrame.from_dict(sent, orient='index')

In [33]:
conn = sqlite3.connect('sent.db')
sent.to_sql('sent', conn)

## 2. Sentiment Score using pmi

#### 1. Calculate Sentiment Score using pmi

In [505]:
conn = sqlite3.connect('sent.db')
temp = pd.read_sql('SELECT * FROM sent', conn)

In [506]:
from collections import defaultdict
sent = defaultdict(lambda:0)
for _ in range(len(temp)):
    sent[temp['index'][_]] = temp['0'][_]

In [522]:
from soynlp.word import WordExtractor
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.vectorizer import sent_to_word_contexts_matrix
from soynlp.word import pmi as pmi_func
from soynlp.utils import most_similar
from soynlp.tokenizer import LTokenizer

score_dict = defaultdict(lambda:0)
word_extractor = WordExtractor()
words = {_:1.0 for _ in sent.keys()}
sentiment = pd.read_excel('sentiment.xlsx')
for k,v in sent.items():
    word_extractor.train([v])
    cohesions = word_extractor.all_cohesion_scores()
    l_cohesions = {word:score[0] for word, score in cohesions.items()}
    l_cohesions.update(words)
    tokenizer = LTokenizer(l_cohesions)
    x, idx2vocab = sent_to_word_contexts_matrix([v], windows = 3, min_tf = 10, tokenizer=tokenizer, dynamic_weight = False, verbose = True)
    pmi, px, py = pmi_func(x, min_pmi = 0, alpha = 0.0, beta = 0.75)
    vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
    query = vocab2idx[k]
    submatrix = pmi[query,:].tocsr() # get the row of query
    contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
    pmi_i = submatrix.data
    most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
    most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:10]
    most_relateds = [(idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds if len(idx2vocab[idx]) > 1]
    pn_score = 0
    for _ in most_relateds:
        if _[0] in ' '.join(sentiment.iloc[:,1]):
            if sentiment[sentiment[0]==_[0]]['P/N'].iloc[0] != 0:
                pn_score += _[1] * sentiment[sentiment[0]==_[0]]['P/N'].iloc[0]
    score_dict[k] = pn_score

training was done. used memory 1.159 Gb1.159 Gb
all cohesion probabilities was computed. # words = 627
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.159 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.159 Gb
  - (word, context) matrix was constructed. shape = (112, 112)                    
  - done


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [329]:
temp = pd.DataFrame(sorted(score_dict.items(), key=lambda _:_[1], reverse=True))
temp.to_excel('sentiment_result.xlsx')

#### 2. Update Sentiment Dictionary using Sentiment Score

In [330]:
import pandas as pd
import re
sentiment_result = pd.read_excel('sentiment_result.xlsx')

In [367]:
sent_dict = defaultdict(lambda:0)
for _ in range(len(sentiment_result)):
    if re.search(r'(-*\d[.]\d+)',str(sentiment_result[1][_])):
        if float(re.search(r'(-*\d[.]\d+)',str(sentiment_result[1][_])).group(1)) > 4:
            sent_dict[sentiment_result[0][_]] = "P"
        elif float(re.search(r'(-*\d[.]\d+)',str(sentiment_result[1][_])).group(1)) < -4.5:
            sent_dict[sentiment_result[0][_]] = "N"
sent_dict = pd.DataFrame.from_dict(sent_dict, orient='index')
positive = sent_dict[sent_dict[0] == 'P'].index
negative = sent_dict[sent_dict[0] == 'N'].index

In [370]:
conn = sqlite3.connect('sent.db')
sent = pd.read_sql('SELECT * FROM sent', conn)
ptemp = pd.DataFrame()
ntemp = pd.DataFrame()
for _ in positive:
    ptemp = pd.concat([ptemp, sent[sent['index'] == _]], axis=0, ignore_index=True)
for _ in negative:
    ntemp = pd.concat([ntemp, sent[sent['index'] == _]], axis=0, ignore_index=True)
from collections import defaultdict
psent = defaultdict(lambda:0)
nsent = defaultdict(lambda:0)
for _ in range(len(ptemp)):
    psent[ptemp['index'][_]] = ptemp['0'][_]
for _ in range(len(ntemp)):
    nsent[ntemp['index'][_]] = ntemp['0'][_]

In [414]:
temp_score_dict = defaultdict(lambda:0)
for k,v in psent.items():
    word_extractor.train([v])
    cohesions = word_extractor.all_cohesion_scores()
    l_cohesions = {word:score[0] for word, score in cohesions.items()}
    l_cohesions.update(words)
    tokenizer = LTokenizer(l_cohesions)
    x, idx2vocab = sent_to_word_contexts_matrix([v], windows = 3, min_tf = 10, tokenizer=tokenizer, dynamic_weight = False, verbose = True)
    pmi, px, py = pmi_func(x, min_pmi = 0, alpha = 0.0, beta = 0.75)
    vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
    query = vocab2idx[k]
    submatrix = pmi[query,:].tocsr() # get the row of query
    contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
    pmi_i = submatrix.data
    most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
    most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:20]
    most_relateds = [(idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds if len(idx2vocab[idx]) > 1]
    for _ in most_relateds:
        temp_score_dict[_[0]] += 0.5
for k,v in nsent.items():
    word_extractor.train([v])
    cohesions = word_extractor.all_cohesion_scores()
    l_cohesions = {word:score[0] for word, score in cohesions.items()}
    l_cohesions.update(words)
    tokenizer = LTokenizer(l_cohesions)
    x, idx2vocab = sent_to_word_contexts_matrix([v], windows = 3, min_tf = 10, tokenizer=tokenizer, dynamic_weight = False, verbose = True)
    pmi, px, py = pmi_func(x, min_pmi = 0, alpha = 0.0, beta = 0.75)
    vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
    query = vocab2idx[k]
    submatrix = pmi[query,:].tocsr() # get the row of query
    contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
    pmi_i = submatrix.data
    most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
    most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:20]
    most_relateds = [(idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds if len(idx2vocab[idx]) > 1]
    for _ in most_relateds:
        temp_score_dict[_[0]] -= 0.5

training was done. used memory 0.811 Gb0.811 Gb
all cohesion probabilities was computed. # words = 169744
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.821 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.821 Gb
  - (word, context) matrix was constructed. shape = (144, 144)                    
  - done
training was done. used memory 0.821 Gb0.821 Gb
all cohesion probabilities was computed. # words = 169744
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.835 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.835 Gb
  - (word, context) matrix was constructed. shape = (53, 53)                    
  - done
training was done. used memory 0.835 Gb0.835 Gb
all cohesion probabilities was computed. # words = 169744
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.835 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.835 Gb
  - (word, context) matrix was constructed. shape = (224

In [416]:
temp = defaultdict(lambda:0)
for _ in range(len(sentiment)):
    temp[sentiment[0][_]] = sentiment['P/N'][_]
for k, v in temp.items():
    for tk,tv in temp_score_dict.items():
        if k == tk:
            temp[k] += tv

In [430]:
test = pd.DataFrame.from_dict(temp, orient='index')
test1 = pd.DataFrame(test.index)
test2 = pd.DataFrame(test[0].values, columns = ['P/N'])
pd.concat([test1,test2], axis=1).to_excel('sentiment.xlsx')

- Function

In [557]:
from collections import defaultdict
from soynlp.word import WordExtractor
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.vectorizer import sent_to_word_contexts_matrix
from soynlp.word import pmi as pmi_func
from soynlp.utils import most_similar
from soynlp.tokenizer import LTokenizer
import pandas as pd
import re

def cal_score():
    mapping_most_related = defaultdict(lambda:0)
    conn = sqlite3.connect('sent.db')
    temp = pd.read_sql('SELECT * FROM sent', conn)
    sent = defaultdict(lambda:0)
    
    for _ in range(len(temp)):
        sent[temp['index'][_]] = temp['0'][_]
    score_dict = defaultdict(lambda:0)
    word_extractor = WordExtractor()
    words = {_:1.0 for _ in sent.keys()}
    sentiment = pd.read_excel('sentiment.xlsx')
    
    for k,v in sent.items():
        word_extractor.train([v])
        cohesions = word_extractor.all_cohesion_scores()
        l_cohesions = {word:score[0] for word, score in cohesions.items()}
        l_cohesions.update(words)
        tokenizer = LTokenizer(l_cohesions)
        x, idx2vocab = sent_to_word_contexts_matrix([v], windows = 3, min_tf = 10, tokenizer=tokenizer, dynamic_weight = False, verbose = True)
        pmi, px, py = pmi_func(x, min_pmi = 0, alpha = 0.0, beta = 0.75)
        vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
        query = vocab2idx[k]
        submatrix = pmi[query,:].tocsr() # get the row of query
        contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
        pmi_i = submatrix.data
        most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
        most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:10]
        most_relateds = [(idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds if len(idx2vocab[idx]) > 1]
        mapping_most_related[k] = most_relateds
        pn_score = 0
        for _ in most_relateds:
            if _[0] in ' '.join(sentiment.iloc[:,1]):
                if sentiment[sentiment[0]==_[0]]['P/N'].iloc[0] != 0:
                    pn_score += _[1] * sentiment[sentiment[0]==_[0]]['P/N'].iloc[0]
        score_dict[k] = pn_score
        
    temp = pd.DataFrame(sorted(score_dict.items(), key=lambda _:_[1], reverse=True))
    temp.to_excel('sentiment_result.xlsx')
    
    return mapping_most_related
    
def update_score(positive, negative):
    from collections import defaultdict
    sentiment_result = pd.read_excel('sentiment_result.xlsx')
    sent_dict = defaultdict(lambda:0)
    
    for _ in range(len(sentiment_result)):
        if re.search(r'(-*\d[.]\d+)',str(sentiment_result[1][_])):
            if float(re.search(r'(-*\d[.]\d+)',str(sentiment_result[1][_])).group(1)) > 4 and sentiment_result[0][_] not in positive:
                sent_dict[sentiment_result[0][_]] = "P"
            elif float(re.search(r'(-*\d[.]\d+)',str(sentiment_result[1][_])).group(1)) < -4.5 and sentiment_result[0][_] not in negative:
                sent_dict[sentiment_result[0][_]] = "N"
    sent_dict = pd.DataFrame.from_dict(sent_dict, orient='index')
    positive = sent_dict[sent_dict[0] == 'P'].index
    negative = sent_dict[sent_dict[0] == 'N'].index
    conn = sqlite3.connect('sent.db')
    sent = pd.read_sql('SELECT * FROM sent', conn)
    ptemp = pd.DataFrame()
    ntemp = pd.DataFrame()
    for _ in positive:
        ptemp = pd.concat([ptemp, sent[sent['index'] == _]], axis=0, ignore_index=True)
    for _ in negative:
        ntemp = pd.concat([ntemp, sent[sent['index'] == _]], axis=0, ignore_index=True)

    psent = defaultdict(lambda:0)
    nsent = defaultdict(lambda:0)
    for _ in range(len(ptemp)):
        psent[ptemp['index'][_]] = ptemp['0'][_]
    for _ in range(len(ntemp)):
        nsent[ntemp['index'][_]] = ntemp['0'][_]
    temp_score_dict = defaultdict(lambda:0)
    
    for k,v in psent.items():
        word_extractor.train([v])
        cohesions = word_extractor.all_cohesion_scores()
        l_cohesions = {word:score[0] for word, score in cohesions.items()}
        l_cohesions.update(words)
        tokenizer = LTokenizer(l_cohesions)
        x, idx2vocab = sent_to_word_contexts_matrix([v], windows = 3, min_tf = 10, tokenizer=tokenizer, dynamic_weight = False, verbose = True)
        pmi, px, py = pmi_func(x, min_pmi = 0, alpha = 0.0, beta = 0.75)
        vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
        query = vocab2idx[k]
        submatrix = pmi[query,:].tocsr() # get the row of query
        contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
        pmi_i = submatrix.data
        most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
        most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:20]
        most_relateds = [(idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds if len(idx2vocab[idx]) > 1]
        for _ in most_relateds:
            temp_score_dict[_[0]] += 0.5
            
    for k,v in nsent.items():
        word_extractor.train([v])
        cohesions = word_extractor.all_cohesion_scores()
        l_cohesions = {word:score[0] for word, score in cohesions.items()}
        l_cohesions.update(words)
        tokenizer = LTokenizer(l_cohesions)
        x, idx2vocab = sent_to_word_contexts_matrix([v], windows = 3, min_tf = 10, tokenizer=tokenizer, dynamic_weight = False, verbose = True)
        pmi, px, py = pmi_func(x, min_pmi = 0, alpha = 0.0, beta = 0.75)
        vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
        query = vocab2idx[k]
        submatrix = pmi[query,:].tocsr() # get the row of query
        contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
        pmi_i = submatrix.data
        most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
        most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:20]
        most_relateds = [(idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds if len(idx2vocab[idx]) > 1]
        for _ in most_relateds:
            temp_score_dict[_[0]] -= 0.5
            
    temp = defaultdict(lambda:0)
    for _ in range(len(sentiment)):
        temp[sentiment[0][_]] = sentiment['P/N'][_]
    for k, v in temp.items():
        for tk,tv in temp_score_dict.items():
            if k == tk:
                temp[k] += tv
    test = pd.DataFrame.from_dict(temp, orient='index')
    test1 = pd.DataFrame(test.index)
    test2 = pd.DataFrame(test[0].values, columns = ['P/N'])
    pd.concat([test1,test2], axis=1).to_excel('sentiment.xlsx')
    
    return positive, negative

In [529]:
mapping_most_related = cal_score()

training was done. used memory 1.177 Gb1.177 Gb
all cohesion probabilities was computed. # words = 627
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.177 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.177 Gb
  - (word, context) matrix was constructed. shape = (112, 112)                    
  - done
training was done. used memory 1.177 Gb1.177 Gb
all cohesion probabilities was computed. # words = 864
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.177 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.177 Gb
  - (word, context) matrix was constructed. shape = (52, 52)                    
  - done
training was done. used memory 1.177 Gb1.177 Gb
all cohesion probabilities was computed. # words = 1001
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.177 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.177 Gb
  - (word, context) matrix was constructed. shape = (41, 41)    

training was done. used memory 1.187 Gb1.187 Gb
all cohesion probabilities was computed. # words = 8300
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.187 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.187 Gb
  - (word, context) matrix was constructed. shape = (21, 21)                    
  - done
training was done. used memory 1.187 Gb1.187 Gb
all cohesion probabilities was computed. # words = 8438
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.187 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.187 Gb
  - (word, context) matrix was constructed. shape = (69, 69)                    
  - done
training was done. used memory 1.187 Gb1.188 Gb
all cohesion probabilities was computed. # words = 9020
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.187 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.187 Gb
  - (word, context) matrix was constructed. shape = (301, 301)  

training was done. used memory 1.207 Gb1.207 Gb
all cohesion probabilities was computed. # words = 18652
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.207 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.207 Gb
  - (word, context) matrix was constructed. shape = (27, 27)                    
  - done
training was done. used memory 1.207 Gb1.207 Gb
all cohesion probabilities was computed. # words = 18673
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.207 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.207 Gb
  - (word, context) matrix was constructed. shape = (16, 16)                    
  - done
training was done. used memory 1.207 Gb1.207 Gb
all cohesion probabilities was computed. # words = 18731
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.207 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.207 Gb
  - (word, context) matrix was constructed. shape = (51, 51) 

training was done. used memory 1.210 Gb1.210 Gb
all cohesion probabilities was computed. # words = 22296
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.210 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.210 Gb
  - (word, context) matrix was constructed. shape = (67, 67)                    
  - done
training was done. used memory 1.210 Gb1.210 Gb
all cohesion probabilities was computed. # words = 22352
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.210 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.210 Gb
  - (word, context) matrix was constructed. shape = (62, 62)                    
  - done
training was done. used memory 1.210 Gb1.210 Gb
all cohesion probabilities was computed. # words = 22472
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.210 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.210 Gb
  - (word, context) matrix was constructed. shape = (82, 82) 

training was done. used memory 1.210 Gb1.217 Gb
all cohesion probabilities was computed. # words = 26452
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.210 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.210 Gb
  - (word, context) matrix was constructed. shape = (60, 60)                    
  - done
training was done. used memory 1.224 Gb1.227 Gb
all cohesion probabilities was computed. # words = 30852
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.229 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.230 Gb
  - (word, context) matrix was constructed. shape = (1856, 1856)                    
  - done
training was done. used memory 1.235 Gb1.235 Gb
all cohesion probabilities was computed. # words = 30876
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=1.235 Gb
  - scanning (word, context) pairs from 0 sents, mem=1.235 Gb
  - (word, context) matrix was constructed. shape = (69, 

...

all cohesion probabilities was computed. # words = 158997
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.916 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.916 Gb
  - (word, context) matrix was constructed. shape = (59, 59)                    
  - done
training was done. used memory 0.916 Gb0.916 Gb
all cohesion probabilities was computed. # words = 159137
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.916 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.916 Gb
  - (word, context) matrix was constructed. shape = (160, 160)                    
  - done
training was done. used memory 0.916 Gb0.916 Gb
all cohesion probabilities was computed. # words = 159142
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.916 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.916 Gb
  - (word, context) matrix was constructed. shape = (21, 21)                    
  - done
training was d

training was done. used memory 0.916 Gb0.916 Gb
all cohesion probabilities was computed. # words = 160791
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.916 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.916 Gb
  - (word, context) matrix was constructed. shape = (60, 60)                    
  - done
training was done. used memory 0.916 Gb0.916 Gb
all cohesion probabilities was computed. # words = 160836
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.916 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.916 Gb
  - (word, context) matrix was constructed. shape = (42, 42)                    
  - done
training was done. used memory 0.916 Gb0.916 Gb
all cohesion probabilities was computed. # words = 160878
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.916 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.916 Gb
  - (word, context) matrix was constructed. shape = (54, 5

training was done. used memory 0.719 Gb0.719 Gb
all cohesion probabilities was computed. # words = 164308
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.719 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.719 Gb
  - (word, context) matrix was constructed. shape = (242, 242)                    
  - done
training was done. used memory 0.719 Gb0.719 Gb
all cohesion probabilities was computed. # words = 164375
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.719 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.719 Gb
  - (word, context) matrix was constructed. shape = (65, 65)                    
  - done
training was done. used memory 0.719 Gb0.719 Gb
all cohesion probabilities was computed. # words = 164441
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.719 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.719 Gb
  - (word, context) matrix was constructed. shape = (100

training was done. used memory 0.721 Gb0.721 Gb
all cohesion probabilities was computed. # words = 166921
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.721 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.721 Gb
  - (word, context) matrix was constructed. shape = (928, 928)                    
  - done
training was done. used memory 0.721 Gb0.721 Gb
all cohesion probabilities was computed. # words = 166923
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.721 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.721 Gb
  - (word, context) matrix was constructed. shape = (62, 62)                    
  - done
training was done. used memory 0.721 Gb0.721 Gb
all cohesion probabilities was computed. # words = 167027
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.721 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.721 Gb
  - (word, context) matrix was constructed. shape = (86,

training was done. used memory 0.713 Gb0.713 Gb
all cohesion probabilities was computed. # words = 169395
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.713 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.713 Gb
  - (word, context) matrix was constructed. shape = (74, 74)                    
  - done
training was done. used memory 0.713 Gb0.713 Gb
all cohesion probabilities was computed. # words = 169655
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.713 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.713 Gb
  - (word, context) matrix was constructed. shape = (374, 374)                    
  - done
training was done. used memory 0.713 Gb0.713 Gb
all cohesion probabilities was computed. # words = 169744
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.713 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.713 Gb
  - (word, context) matrix was constructed. shape = (124

In [ ]:
from tqdm.notebook import tqdm
positive = []
negative = []
for _ in tqdm(range(100)):
    score_dict = defaultdict(lambda:0)
    sentiment = pd.read_excel('sentiment.xlsx')
    for k,v in mapping_most_related.items():
        pn_score = 0
        for _ in v:
            if _[0] in ' '.join(sentiment.iloc[:,1]):
                pn_score += _[1] * sentiment[sentiment[0]==_[0]]['P/N'].iloc[0]
        score_dict[k] = pn_score
    temp = pd.DataFrame(sorted(score_dict.items(), key=lambda _:_[1], reverse=True))
    temp.to_excel('sentiment_result.xlsx')
    positive, negative = update_score(positive, negative)

training was done. used memory 0.785 Gb0.790 Gb
all cohesion probabilities was computed. # words = 33189
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.788 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.788 Gb
  - (word, context) matrix was constructed. shape = (141, 141)                    
  - done
training was done. used memory 0.788 Gb0.793 Gb
all cohesion probabilities was computed. # words = 33189
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.788 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.788 Gb
  - (word, context) matrix was constructed. shape = (50, 50)                    
  - done
training was done. used memory 0.788 Gb0.793 Gb
all cohesion probabilities was computed. # words = 33189
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.788 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.788 Gb
  - (word, context) matrix was constructed. shape = (233, 2

training was done. used memory 0.877 Gb0.882 Gb
all cohesion probabilities was computed. # words = 33189
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.879 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.879 Gb
  - (word, context) matrix was constructed. shape = (142, 142)                    
  - done
training was done. used memory 0.880 Gb0.891 Gb
all cohesion probabilities was computed. # words = 33189
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.882 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.881 Gb
  - (word, context) matrix was constructed. shape = (2345, 2345)                    
  - done
training was done. used memory 0.882 Gb0.888 Gb
all cohesion probabilities was computed. # words = 33189
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.882 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.882 Gb
  - (word, context) matrix was constructed. shape = (44

training was done. used memory 0.887 Gb0.887 Gb
all cohesion probabilities was computed. # words = 37203
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.887 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.887 Gb
  - (word, context) matrix was constructed. shape = (77, 77)                    
  - done
training was done. used memory 0.887 Gb0.888 Gb
all cohesion probabilities was computed. # words = 37833
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.887 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.887 Gb
  - (word, context) matrix was constructed. shape = (471, 471)                    
  - done
training was done. used memory 0.887 Gb0.887 Gb
all cohesion probabilities was computed. # words = 37885
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.887 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.887 Gb
  - (word, context) matrix was constructed. shape = (69, 69

training was done. used memory 0.888 Gb0.888 Gb
all cohesion probabilities was computed. # words = 40917
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.888 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.888 Gb
  - (word, context) matrix was constructed. shape = (60, 60)                    
  - done
training was done. used memory 0.888 Gb0.888 Gb
all cohesion probabilities was computed. # words = 41061
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.888 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.888 Gb
  - (word, context) matrix was constructed. shape = (106, 106)                    
  - done
training was done. used memory 0.888 Gb0.890 Gb
all cohesion probabilities was computed. # words = 42421
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.888 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.888 Gb
  - (word, context) matrix was constructed. shape = (841, 8

training was done. used memory 0.939 Gb0.939 Gb
all cohesion probabilities was computed. # words = 69856
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.939 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.939 Gb
  - (word, context) matrix was constructed. shape = (122, 122)                    
  - done
training was done. used memory 0.939 Gb0.939 Gb
all cohesion probabilities was computed. # words = 69924
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.939 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.939 Gb
  - (word, context) matrix was constructed. shape = (60, 60)                    
  - done
training was done. used memory 0.939 Gb0.939 Gb
all cohesion probabilities was computed. # words = 69984
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.939 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.939 Gb
  - (word, context) matrix was constructed. shape = (45, 45

...

training was done. used memory 0.787 Gb0.787 Gb
all cohesion probabilities was computed. # words = 146913
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.787 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.787 Gb
  - (word, context) matrix was constructed. shape = (596, 596)                    
  - done
training was done. used memory 0.768 Gb0.768 Gb
all cohesion probabilities was computed. # words = 146913
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.768 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.768 Gb
  - (word, context) matrix was constructed. shape = (98, 98)                    
  - done
training was done. used memory 0.768 Gb0.768 Gb
all cohesion probabilities was computed. # words = 146913
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.768 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.768 Gb
  - (word, context) matrix was constructed. shape = (269

training was done. used memory 0.768 Gb0.768 Gb
all cohesion probabilities was computed. # words = 148261
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.768 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.768 Gb
  - (word, context) matrix was constructed. shape = (80, 80)                    
  - done
training was done. used memory 0.768 Gb0.768 Gb
all cohesion probabilities was computed. # words = 148501
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.768 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.768 Gb
  - (word, context) matrix was constructed. shape = (311, 311)                    
  - done
training was done. used memory 0.768 Gb0.768 Gb
all cohesion probabilities was computed. # words = 148512
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.768 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.768 Gb
  - (word, context) matrix was constructed. shape = (37,

  - scanning (word, context) pairs from 0 sents, mem=0.723 Gb
  - (word, context) matrix was constructed. shape = (41, 41)                    
  - done
training was done. used memory 0.723 Gb0.723 Gb
all cohesion probabilities was computed. # words = 149515
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.723 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.723 Gb
  - (word, context) matrix was constructed. shape = (95, 95)                    
  - done
training was done. used memory 0.723 Gb0.723 Gb
all cohesion probabilities was computed. # words = 149598
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.723 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.723 Gb
  - (word, context) matrix was constructed. shape = (160, 160)                    
  - done
training was done. used memory 0.723 Gb0.723 Gb
all cohesion probabilities was computed. # words = 149598
Create (word, contexts) matrix
  - counting word fre

Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.728 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.728 Gb
  - (word, context) matrix was constructed. shape = (870, 870)                    
  - done
training was done. used memory 0.730 Gb0.730 Gb
all cohesion probabilities was computed. # words = 150151
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.730 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.730 Gb
  - (word, context) matrix was constructed. shape = (95, 95)                    
  - done
training was done. used memory 0.730 Gb0.730 Gb
all cohesion probabilities was computed. # words = 150370
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.730 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.730 Gb
  - (word, context) matrix was constructed. shape = (297, 297)                    
  - done
training was done. used memory 0.730 Gb0.730 Gb
all cohesion probabili

training was done. used memory 0.750 Gb0.750 Gb
all cohesion probabilities was computed. # words = 151462
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.750 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.750 Gb
  - (word, context) matrix was constructed. shape = (376, 376)                    
  - done
training was done. used memory 0.750 Gb0.750 Gb
all cohesion probabilities was computed. # words = 151462
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.750 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.750 Gb
  - (word, context) matrix was constructed. shape = (38, 38)                    
  - done
training was done. used memory 0.750 Gb0.750 Gb
all cohesion probabilities was computed. # words = 151509
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.750 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.750 Gb
  - (word, context) matrix was constructed. shape = (67,